In [ ]:
!pip install openai python-dotenv pypdf gradio

In [3]:
from dotenv import load_dotenv
from IPython.display import Markdown
from openai import OpenAI
from pypdf import PdfReader
import os
import gradio as gr

In [4]:
load_dotenv(override=True)

True

In [5]:
openaiapi = os.getenv("OPENAI_API_KEY")
groqapi = os.getenv("groq_api_key")

In [6]:
if openaiapi:
    print(f"Openai key is found and starts with: {openaiapi[:8]}")
else:
    print("Open ai api not found")

if groqapi:
    print(f"groqapi key is found and starts with: {groqapi[:8]}")
else:
    print("groqapi ai api not found")

Openai key is found and starts with: sk-proj-
groqapi key is found and starts with: gsk_Vopn


In [7]:
# System prompt tailored for property recommendations
system_prompt_property = """
You are a real estate advisor AI. 
You are given a dataset of properties including location, price, bedrooms, bathrooms, parking spaces, toilets, title, and description. 
When a user provides their preferences (budget, location, property type, number of bedrooms), you must return the top 3 properties that best match their needs. 
Provide a short reasoning for each recommendation. 
Always consider price affordability and feature match.
"""


In [8]:
processed_path = "C:/Users/Admin/projects/Property-Evaluator-Agent/data/processed/cleaned_property_data.csv"

In [17]:

import pandas as pd

# Ensure df_cleaned is a DataFrame
processed_path = "C:/Users/Admin/projects/Property-Evaluator-Agent/data/processed/cleaned_property_data.csv"
df_cleaned = pd.read_csv(processed_path)


In [18]:
df_cleaned

,Location,Currency,Price,Contact,Bedrooms,Realtor,Bathrooms,Title,Description,Parking Spaces,Toilets,Score
0,"Urban Prime Three Phase 2 Estate, Ogombo, Ajah...",₦,1.500000e+08,8067495852,5.0,Lake Pad Ventures,6.0,Luxury 5 Bedrooms Semi-detached Duplex,Hurry now!\nfor sale\nluxury 5 bedroom on two ...,5.0,7.0,22.0
1,"Ikoyi, Lagos",₦,4.290552e+09,7065266133,6.0,Bera Deals,6.0,Luxury 6 Bedrooms Penthouse Condo,Direct brief from the owner\n- the detailed vi...,0.0,7.0,21.5
2,"Pinnock Beach Estate, Osapa, Lekki, Lagos",₦,1.400000e+09,7065266133,6.0,Bera Deals,4.0,Newly Built Massive House / Mansion,Direct brief / direct instruction from the own...,0.0,7.0,19.5
3,"Oseni Mayegun Street, Ologolo, Lekki, Lagos",₦,2.000000e+08,8079792003,4.0,Paul Osaji And Co,4.0,4 Bedroom Detached House With 2 Mini Flats Bq,For sale in ologolo\n4 bedroom fully detached ...,0.0,4.0,14.0
4,"Ikate Lekki, Ikate, Lekki, Lagos",₦,2.100000e+08,8084677480,5.0,Jasmine Real Properties And Estate Services,5.0,Super Luxury 5bed Semi Detach Duplex With A Bq,5 bed ikate n215m\nluxury 5 bedroom semidetach...,0.0,6.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...
50242,"Tulip Heaven, Chevron, Lekki Phase 1, Lekki, L...",₦,2.600000e+08,8033012341,5.0,First Close,6.0,Newly Built 5 Bedrooms Fully Detached House,"Holiday price now n260,000,000.00 from initial...",0.0,7.0,19.5
50243,"Aguda, Surulere, Lagos",₦,1.500000e+08,8054563213,4.0,Harris Consult Realty,4.0,Renovated Block Of 16 Flats,Direct brief\nfor sale\n16 flat comprised of 9...,0.0,5.0,14.5
50244,"Katampe Extension, Katampe, Abuja",₦,2.500000e+09,8155321626,9.0,Akerejolabisoye Global Ltd,10.0,Newly Built Luxury 9 Bedroom Mansion With 2 Be...,Newly built luxury 9 bedroom mansion with 2 be...,10.0,10.0,38.0
50245,"Banana Island, Ikoyi, Lagos",₦,6.129360e+09,9016641836,5.0,Godmaan Global Realestate,8.0,"Convenient, Contemporary Living With Treetop V...","Located in a sought-after neighborhood, this p...",8.0,8.0,26.0


In [12]:
openai = OpenAI()

In [13]:


def chat_property(user_prompt, history=[]):
    messages = [{"role": "system", "content": system_prompt_property}] + history + [{"role": "user", "content": user_prompt}]
    
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    
    reply = response.choices[0].message.content
    return reply


In [ ]:
# Example user input
user_prompt = get_recommendation_input(
    budget=200000000, 
    location="Lagos", 
    property_type="Apartment", 
    bedrooms=3
)

# Generate GPT-4 recommendation
reply = chat_property(user_prompt)
print(reply)


In [19]:
import gradio as gr
from openai import OpenAI

processed_path = "C:/Users/Admin/projects/Property-Evaluator-Agent/data/processed/cleaned_property_data.csv"
df_cleaned = pd.read_csv(processed_path)
# Your existing functions
client = OpenAI()

def get_recommendation_input(budget=None, location=None, property_type=None, bedrooms=None):
    """
    Prepare user query and filtered dataset subset for LLM
    """
    df_filtered = df_cleaned  # assume df_cleaned is globally available

    # Filters
    if budget is not None:
        df_filtered = df_filtered[df_filtered['Price'] <= budget]
    if location:
        df_filtered = df_filtered[df_filtered['Location'].str.contains(location, case=False, na=False)]
    if property_type:
        df_filtered = df_filtered[df_filtered['Title'].str.contains(property_type, case=False, na=False)]
    if bedrooms is not None:
        df_filtered = df_filtered[df_filtered['Bedrooms'] == bedrooms]

    # Select top 5
    df_top = df_filtered.sort_values(by='Score', ascending=False).head(5)

    # Build user prompt
    prompt = f"Recommend top properties based on these preferences:\nBudget: {budget}\n"
    prompt += f"Location: {location}\nProperty Type: {property_type}\nBedrooms: {bedrooms}\n\n"
    prompt += "Available properties:\n"
    for idx, row in df_top.iterrows():
        prompt += f"- {row['Title']} | Price: ₦{row['Price']:,.0f} | Location: {row['Location']} | Bedrooms: {row['Bedrooms']} | Bathrooms: {row['Bathrooms']} | Parking: {row['Parking Spaces']} | Toilets: {row['Toilets']} | Score: {row['Score']}\n"
    return prompt


def chat_property(user_message, history):
    """
    Chat wrapper that keeps history and calls GPT
    """
    # Convert Gradio history format [(user, bot), ...] to OpenAI messages
    messages = [{"role": "system", "content": system_prompt_property}]
    for user, bot in history:
        messages.append({"role": "user", "content": user})
        messages.append({"role": "assistant", "content": bot})
    messages.append({"role": "user", "content": user_message})

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    reply = response.choices[0].message.content
    return reply


# Function to link filters + chat
def recommend_with_filters(budget, location, property_type, bedrooms, history=[]):
    user_prompt = get_recommendation_input(budget, location, property_type, bedrooms)
    reply = chat_property(user_prompt, history)
    history.append((user_prompt, reply))
    return history, history


# Build Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("## 🏡 Property Finder Chatbot")

    with gr.Row():
        budget = gr.Number(label="Budget (₦)", value=200000000)
        location = gr.Textbox(label="Location", placeholder="e.g., Lagos")
        property_type = gr.Textbox(label="Property Type", placeholder="e.g., Apartment")
        bedrooms = gr.Number(label="Bedrooms", value=3)

    chatbot = gr.Chatbot()
    state = gr.State([])

    send_button = gr.Button("Find Properties")

    send_button.click(
        recommend_with_filters,
        inputs=[budget, location, property_type, bedrooms, state],
        outputs=[chatbot, state]
    )

demo.launch()


C:\Users\Admin\AppData\Local\Temp\ipykernel_12820\918194990.py:74: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
